Plan:
* Use linspace to produce a bunch of values at some intervals
* SMARTLY restrict values
* Try and make it make sense
* Code equations first!
* Could brute force with REAL values (restrict to range) or CALCULATE values (use equations from geometry)

## Equations
With credit to tables from _Design of Rockets and Space Launch Vehicles_
* Change in velocity: $$\Delta v = v_{eq}ln(\frac{m_0}{m_f}) = v_{eq}ln(mass\ ratio) = v_{eq}ln(\mu) = g_0 I_{sp}ln\mu$$
    * $g_0$ is Earth's standard gravity at sea level. Taken to be 9.80665 $m/s^2$
* Missing time to burn equation
* Mass fraction: $$\mu = \frac{initial\ mass}{final\ mass} = \frac{m_0}{m_f} = \frac{m_0}{m_0-m_p}$$ 
<br />
* Total Launch Weight (Propellant and shell - NOT payload):
$$W_L = 0.04 Ld^2$$ 
    * NOTE: lb, in, in
    * Can be used to calculate masses instead of using values
<br /><br />
* Body Structure vs. Launch Weight: $$W_{Body Structure}/W_{Launch Weight} = 0.22$$
    * Launch weight includes the propellant
    * Combine with Total launch weight equations

### SM3 Mass Fractions (Mf/Mo)
* First Stage: .376
* Second Stage: .612
* Third Stage: .604

### Range of Real Mass Fractions (Mf/Mo)
With credit to tables from _Design of Rockets and Space Launch Vehicles_
* First Stage: 
    * 0.0432 -> 0.1198
* Second Stage:
    * 0.0495 -> 0.1700
* Third Stage:
    * 0.0824 -> 0.1008

### Range of Isps
With credit to tables from _Design of Rockets and Space Launch Vehicles_
* 200 seconds to 250 seconds 
    * 250 seconds more applicable

In [2]:
import numpy as np


In [27]:
specific_impulse = 250 # seconds
g = 9.80665 # m / s^2
v_eq = g*specific_impulse

meters_to_inches = 39.3700787402 ## multiply to meters
kg_to_lbs = 2.20462

payload = 250 * kg_to_lbs  ## in lbs


## Calculating Delta V using range of actual values

In [4]:
#m0 / mf
mass_frac_first = 1/np.linspace(0.0400, 0.1200)
mass_frac_second = 1/np.linspace(.0450, 0.1700) 
mass_frac_third = 1/np.linspace(.0800, 0.1010)

In [5]:
max_mf1 = np.max(mass_frac_first)
max_mf2 = np.max(mass_frac_second)
max_mf3 = np.max(mass_frac_third)

In [6]:
deltav_first = np.log(mass_frac_first) * v_eq
deltav_second = np.log(mass_frac_second) * v_eq
deltav_third = np.log(mass_frac_third) * v_eq
deltav_total_max = np.max(deltav_first)+np.max(deltav_second)+np.max(deltav_third)

In [7]:
print(
"Unitless values.\nMax Mass Fraction\n"
    "\tFirst: ", max_mf1, "\n\tSecond: ", max_mf2, "\n\tThird: ", max_mf3)
print(
"\n\nUnits in meters per second.\nMax Delta V\n",
    "\tFirst: ", np.max(deltav_first), "\n\tSecond: ", np.max(deltav_second), "\n\tThird: ", np.max(deltav_third),
"\n\nTotal Delta V: ", deltav_total_max)

Unitless values.
Max Mass Fraction
	First:  25.0 
	Second:  22.22222222222222 
	Third:  12.5


Units in meters per second.
Max Delta V
 	First:  7891.597151985934 
	Second:  7602.832900331016 
	Third:  6192.234202426388 

Total Delta V:  21686.66425474334


## Calculating Delta V with Mass Equations

In [8]:
## Constants given in project

total_length = 10.0 * 39.3700787402
diameter = 1.0 * 39.3700787402 ## convert given meters to inches (final unit inches)

In [9]:
launch_weight = 0.04 * total_length * (diameter)*(diameter) ## pounds

In [10]:
print(launch_weight, "lbs")
print(launch_weight/2.205, "kg")

24409.497637972003 lbs
11070.066955996374 kg


In [11]:
## Lengths in ratio form
l1_ratio = np.linspace(0.1, 1, 100)
l2_ratio = np.linspace(0.1, 1, 100)
print(np.round(l1_ratio*10, 1)) ## lengths corresponding to l1's ratio, rounded to tenths

[ 1.   1.1  1.2  1.3  1.4  1.5  1.5  1.6  1.7  1.8  1.9  2.   2.1  2.2
  2.3  2.4  2.5  2.5  2.6  2.7  2.8  2.9  3.   3.1  3.2  3.3  3.4  3.5
  3.5  3.6  3.7  3.8  3.9  4.   4.1  4.2  4.3  4.4  4.5  4.5  4.6  4.7
  4.8  4.9  5.   5.1  5.2  5.3  5.4  5.5  5.5  5.6  5.7  5.8  5.9  6.
  6.1  6.2  6.3  6.4  6.5  6.5  6.6  6.7  6.8  6.9  7.   7.1  7.2  7.3
  7.4  7.5  7.5  7.6  7.7  7.8  7.9  8.   8.1  8.2  8.3  8.4  8.5  8.5
  8.6  8.7  8.8  8.9  9.   9.1  9.2  9.3  9.4  9.5  9.5  9.6  9.7  9.8
  9.9 10. ]


In [12]:
## Create new data type to hold a tuple in each element of the array (makes adding elements easier)
##length 1 ratio, length 2 ratio, length 3 ratio
dt = np.dtype([('l1r', np.float64), ('l2r', np.float64), ('l3r', np.float64)])

## Test dt to make sure it works as expected

test = np.array([(1,2,3), (1,3,4)], dtype = dt)

print(test)
np.append(test, np.array([(4.,4.,2.)], dtype = dt))


[(1., 2., 3.) (1., 3., 4.)]


array([(1., 2., 3.), (1., 3., 4.), (4., 4., 2.)],
      dtype=[('l1r', '<f8'), ('l2r', '<f8'), ('l3r', '<f8')])

In [13]:
## Find all possible ratio combos between the three stages, put it in an np array of tuples
ratio_list = np.array([], dtype=dt )

for i in l1_ratio:
    for j in l2_ratio:
        if(i+j > 1):
            continue
        else:
            ratio_list = np.append(ratio_list, np.array([(i, j, 1-i-j)], dtype = dt)) 
    
    
    

In [14]:
## convert the np array of tuples to a more useful np array of 3 columns and each row representing a tuple of 
## the ratios
ratio_list_converted = np.array([])
for i in ratio_list:
    ratio_list_converted = np.append(ratio_list_converted, np.array([[i[0], i[1], i[2]]]))
    
ratio_list_converted = np.reshape(ratio_list_converted, (-1, 3)).astype(np.float64)

##np.set_printoptions(threshold=np.inf) - to see ENTIRE list
print(np.round(ratio_list_converted*10, 2), "\n")
print(ratio_list_converted.shape)

## 4005 different combinations!

[[1.   1.   8.  ]
 [1.   1.09 7.91]
 [1.   1.18 7.82]
 ...
 [8.91 1.   0.09]
 [8.91 1.09 0.  ]
 [9.   1.   0.  ]] 

(4005, 3)


In [25]:
##now we calculate masses

launch_weights = 0.04 * (10 * ratio_list_converted * meters_to_inches) * diameter * diameter + payload() ## lbs 

final_masses = launch_weights*.22 + payload ## after losing the propellant

In [16]:
test1 = np.array([[0, 2], [4,5]])
print(np.sum(test1, axis = 1))

[2 9]


In [65]:
## now mass fractions - m0/mf

stage_1_mf = np.sum(launch_weights, axis = 1)/(np.sum(launch_weights[:, 1:2], axis=1) + final_masses[:, 0])

stage_2_mf = np.sum(launch_weights[:, 1:2], axis = 1)/(launch_weights[:, 2] + final_masses[:, 1])

stage_3_mf = launch_weights[:, 2]/final_masses[:, 2]

## Calculate mass fractions by taking initial launch weight / final masses of the lowest stage

mass_fractions = np.stack((stage_1_mf, stage_2_mf, stage_3_mf), axis = 1)

print(mass_fractions.shape)

print(mass_fractions[:, 0])
print(stage_1_mf)

print(mass_fractions[:, 1])
print(stage_2_mf)

print(mass_fractions[:, 2])
print(stage_3_mf)



(4005, 3)
[7.12136758 6.70050973 6.3266198  ... 3.23378076 3.14410566 3.21361608]
[7.12136758 6.70050973 6.3266198  ... 3.23378076 3.14410566 3.21361608]
[0.13050442 0.14246209 0.15462395 ... 2.04804184 2.55328098 2.46421952]
[0.13050442 0.14246209 0.15462395 ... 2.04804184 2.55328098 2.46421952]
[4.29847651 4.29582788 4.29312183 ... 1.33374559 0.81967213 0.81967213]
[4.29847651 4.29582788 4.29312183 ... 1.33374559 0.81967213 0.81967213]


In [86]:
## now delta vs

delta_v = np.log(mass_fractions) * v_eq 
delta_v.shape
## and quickly sum for total delta vs

delta_v = np.sum(delta_v, axis = 1)



In [93]:
## Now find best performing mass fractions

print(delta_v)
print(np.max(delta_v))
print(np.argmax(delta_v))

best_performing_index = np.argmax(delta_v)

print(mass_fractions[best_performing_index, :])


[3395.58253032 3459.66004835 3518.18727715 ... 5341.01373502 4619.07388522
 4585.64122209]
6231.431968068142
3592
[2.21786867 2.1652463  2.64490749]


In [104]:
## What if I want to find the next best performing? The next 5 performing?

sorted_delta_v = np.sort(delta_v)

##reverse it to get descending instead of ascending

sorted_delta_v = sorted_delta_v[::-1]

## now it stores biggest first and next biggest after

print(sorted_delta_v[0:10])

for i in sorted_delta_v[0:100]:
    print(i)
    print(mass_fractions[np.where(delta_v == i)[0], :])



[6231.43196807 6231.13029164 6230.67617753 6229.8593052  6228.76524334
 6227.6987571  6225.59067593 6224.72094572 6222.77919848 6222.37892347]
6231.431968068142
[[2.21786867 2.1652463  2.64490749]]
6231.13029164159
[[2.18453723 2.19801291 2.64490749]]
6230.676177526348
[[2.252233   2.13155199 2.64490749]]
6229.859305195821
[[2.1521928  2.22988963 2.64490749]]
6228.7652433409985
[[2.287679   2.09689002 2.64490749]]
6227.698757096287
[[2.12079219 2.26091222 2.64490749]]
6225.590675934372
[[2.32425855 2.06121811 2.64490749]]
6224.720945717543
[[2.09029468 2.29111456 2.64490749]]
6222.77919848226
[[2.16121821 2.100253   2.78837134]]
6222.378923469012
[[2.12955562 2.13113194 2.78837134]]
6222.243384537045
[[2.19383654 2.06857395 2.78837134]]
6221.531226458943
[[2.27756895 2.24330989 2.47593345]]
6221.187831702854
[[2.24243315 2.27814032 2.47593345]]
6221.116630288581
[[2.09880737 2.1612407  2.78837134]]
6221.031661328636
[[2.3620269  2.02449147 2.64490749]]
6220.991549583088
[[2.06066186 2.